In [44]:
!pip install bs4


In [45]:
!pip install requests

In [46]:
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import time
import random
import pandas as pd


In [47]:
BASE_URL = "https://www.amazon.in/s?"
QUERY = "Laptops"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9"

}

In [48]:
session= requests.Session()
retry = Retry(connect=3,backoff_factor=1,status_forcelist=[429,500,502,503,504])
adapter = HTTPAdapter(max_retries=retry)
session.mount("http://",adapter)
session.mount("https://",adapter)

In [49]:
all_data = []

for page in range(1,20):
  params = {'k': QUERY, 'page': page}
  print(f'scraping page {page}...')

  try:
    response = session.get(BASE_URL, params=params, headers=HEADERS, timeout=10)
    response.raise_for_status()
  except requests.RequestException as e:
    print(f'Error on page {page}: {e}')
    continue

  soup = BeautifulSoup(response.text, 'html.parser')

  # Extract product information for the current page
  for result in soup.find_all('div', {'data-component-type': 's-search-result'}):
    BrandName_tag =result.find('span',{'class':'a-size-base-plus a-color-base'})
    ProductName_tag = result.find('a', {'class':'a-link-normal s-line-clamp-2 s-line-clamp-3-for-col-12 s-link-style a-text-normal'})
    ProductPrice_tag = result.find('span', class_='a-price-whole')
    ProductRating_tag = result.find('span', class_='a-icon-alt')
    ProductNoOfReviews = result.find('span', class_='a-size-mini puis-normal-weight-text s-underline-text')
      
    BrandName = BrandName_tag.get_text(strip=True) if BrandName_tag else None
    ProductName = ProductName_tag.get_text(strip=True) if ProductName_tag else None
    ProductPrice = ProductPrice_tag.get_text(strip=True) if ProductPrice_tag else None
    ProductRating = ProductRating_tag.get_text(strip=True) if ProductRating_tag else None
    productNoOfReview = ProductNoOfReviews.get_text(strip=True) if ProductNoOfReviews else None


    all_data.append({
        "BrandName": BrandName,
        "ProductName": ProductName,
        "ProductPrice": ProductPrice,
        "ProductRating": ProductRating,
        "ProductNoOfReview": productNoOfReview})
  #delay
  time.sleep(random.uniform(2, 5))

print(f"Scraped {len(all_data)} products total")

scraping page 1...
scraping page 2...
scraping page 3...
scraping page 4...
scraping page 5...
scraping page 6...
scraping page 7...
scraping page 8...
scraping page 9...
scraping page 10...
scraping page 11...
scraping page 12...
scraping page 13...
scraping page 14...
scraping page 15...
scraping page 16...
scraping page 17...
scraping page 18...
scraping page 19...
Scraped 418 products total


In [50]:
df = pd.DataFrame(all_data)
display(df.head())

,BrandName,ProductName,ProductPrice,ProductRating,ProductNoOfReview
0,None,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...","91,900",4.6 out of 5 stars,(783)
1,None,"Lenovo IdeaPad Slim 3, Intel Core i3 13th Gen ...","37,990",3.7 out of 5 stars,(9)
2,None,Acer Aspire 3 12th Gen Intel Core i3-1215U Pre...,"29,990",4.0 out of 5 stars,(635)
3,None,Lenovo IdeaPad Slim 3 13th Gen Intel Core i5-1...,"55,861",4.0 out of 5 stars,(318)
4,None,"HP 14 (2025), Intel Core 5 14th Gen 120U - (8 ...","48,490",5.0 out of 5 stars,(1)


In [51]:
df.to_csv("laptops_amazon.csv")

In [52]:
df=pd.read_csv('laptops_amazon.csv')

In [53]:
df['ProductPrice'] = df['ProductPrice'].astype(str).str.strip('₹').str.replace(',', '', regex=False).str.replace('.', '', regex=False)
df['ProductPrice'] = pd.to_numeric(df['ProductPrice'], errors='coerce').astype('Int64')

In [54]:
df["ProductRating"] = (df["ProductRating"].astype(str).str.extract(r"([\d.]+)").astype(float))

In [55]:
df = df.rename(columns={'ProductNoOfReview': 'Reviews'})

In [56]:
def clean_reviews(x):
    if pd.isna(x):
        return None
    x = str(x).strip("()").replace(",", "").strip()
    if x.lower() == "none" or x == "":
        return None
    if "K" in x:
        return int(float(x.replace("K", "")) * 1000)
    elif "M" in x:
        return int(float(x.replace("M", "")) * 1_000_000)
    else:
        try:
            return int(float(x))
        except:
            return None

df["Reviews"] = df["Reviews"].apply(clean_reviews).astype("Int64")

# Ensure 'Rating_Out_of_5' is float
df["ProductRating"] = df["ProductRating"].astype(float)

# Check types and sample output
print(df.dtypes)
print(df.head())

Unnamed: 0         int64
BrandName        float64
ProductName       object
ProductPrice       Int64
ProductRating    float64
Reviews            Int64
dtype: object
   Unnamed: 0  BrandName                                        ProductName  \
0           0        NaN  Apple 2025 MacBook Air (13-inch, Apple M4 chip...   
1           1        NaN  Lenovo IdeaPad Slim 3, Intel Core i3 13th Gen ...   
2           2        NaN  Acer Aspire 3 12th Gen Intel Core i3-1215U Pre...   
3           3        NaN  Lenovo IdeaPad Slim 3 13th Gen Intel Core i5-1...   
4           4        NaN  HP 14 (2025), Intel Core 5 14th Gen 120U - (8 ...   

   ProductPrice  ProductRating  Reviews  
0         91900            4.6      783  
1         37990            3.7        9  
2         29990            4.0      635  
3         55861            4.0      318  
4         48490            5.0        1  


In [57]:

df

,Unnamed: 0,BrandName,ProductName,ProductPrice,ProductRating,Reviews
0,0,NaN,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",91900,4.6,783
1,1,NaN,"Lenovo IdeaPad Slim 3, Intel Core i3 13th Gen ...",37990,3.7,9
2,2,NaN,Acer Aspire 3 12th Gen Intel Core i3-1215U Pre...,29990,4.0,635
3,3,NaN,Lenovo IdeaPad Slim 3 13th Gen Intel Core i5-1...,55861,4.0,318
4,4,NaN,"HP 14 (2025), Intel Core 5 14th Gen 120U - (8 ...",48490,5.0,1
...,...,...,...,...,...,...
413,413,NaN,Mini Laptop Stand for Desk Holder Computer Por...,249,4.4,7
414,414,NaN,acer Flow Wireless Mouse | 1600 DPI Optical Se...,334,4.1,866
415,415,NaN,HARISSONS Laptop & Mobile Travel Friendly Char...,236,4.1,38
416,416,NaN,"Primebook 2 Pro 2025 (New Launch) | 8GB RAM, 1...",18890,4.5,127


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     418 non-null    int64  
 1   BrandName      0 non-null      float64
 2   ProductName    418 non-null    object 
 3   ProductPrice   418 non-null    Int64  
 4   ProductRating  411 non-null    float64
 5   Reviews        411 non-null    Int64  
dtypes: Int64(2), float64(2), int64(1), object(1)
memory usage: 20.5+ KB


In [59]:

df.describe()

,Unnamed: 0,BrandName,ProductPrice,ProductRating,Reviews
count,418.000000,0.0,418.0,411.000000,411.0
mean,208.500000,NaN,35203.578947,4.091484,1867.79562
std,120.810458,NaN,61443.377151,0.497212,6574.969764
min,0.000000,NaN,89.0,2.000000,1.0
25%,104.250000,NaN,386.75,3.800000,54.5
50%,208.500000,NaN,1942.0,4.100000,268.0
75%,312.750000,NaN,55990.0,4.400000,918.5
max,417.000000,NaN,490006.0,5.000000,78000.0


In [60]:
df.to_csv("laptops_amazon_dataset.csv", index=False, encoding="utf-8-sig")
print("\n Cleaned data saved as laptops_amazon_dataset.csv")



 Cleaned data saved as laptops_amazon_dataset.csv


In [61]:
Lap=pd.read_csv("laptops_amazon_dataset.csv")
Lap.head()

,Unnamed: 0,BrandName,ProductName,ProductPrice,ProductRating,Reviews
0,0,NaN,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",91900,4.6,783.0
1,1,NaN,"Lenovo IdeaPad Slim 3, Intel Core i3 13th Gen ...",37990,3.7,9.0
2,2,NaN,Acer Aspire 3 12th Gen Intel Core i3-1215U Pre...,29990,4.0,635.0
3,3,NaN,Lenovo IdeaPad Slim 3 13th Gen Intel Core i5-1...,55861,4.0,318.0
4,4,NaN,"HP 14 (2025), Intel Core 5 14th Gen 120U - (8 ...",48490,5.0,1.0


In [62]:
pip install mysql-connector-python


  Using cached mysql_connector_python-9.5.0-cp312-cp312-win_amd64.whl.metadata (7.7 kB)
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB 991.0 kB/s eta 0:00:17
   ---------------------------------------- 0.1/16.5 MB 2.0 MB/s eta 0:00:09
    --------------------------------------- 0.4/16.5 MB 3.3 MB/s eta 0:00:05
   - -------------------------------------- 0.6/16.5 MB 4.4 MB/s eta 0:00:04
   - -------------------------------------- 0.6/16.5 MB 4.4 MB/s eta 0:00:04
   - -------------------------------------- 0.7/16.5 MB 2.9 MB/s eta 0:00:06
   -- ------------------------------------- 1.0/16.5 MB 3.4 MB/s eta 0:00:05
   --- ------------------------------------ 1.5/16.5 MB 4.4 MB/s eta 0:00:04
   ----- ---------------------------------- 2.4/16.5 MB 6.1 MB/s eta 0:00:03
   ------- -------------------------------- 3.0/16.5 MB 6.9 MB/s eta 0:00:02
   -------- ------------------------------- 3.4/16.5 MB 7.2 MB/s eta 0

In [63]:
import pandas as pd
from sqlalchemy import create_engine

In [66]:
%pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [70]:
import sys
sys.executable


'C:\\Users\\Raagul\\anaconda3\\python.exe'

In [84]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

username = 'root'
password = 'Jesraa@11'          # if special chars, encode below
host = '127.0.0.1'                 # <-- must be only hostname/IP
port = 3306
database = 'lapdata'

# If password contains special characters, URL-encode it:
password_encoded = quote_plus(password)

engine = create_engine(
    f"mysql+pymysql://{username}:{password_encoded}@{host}:{port}/{database}",
    pool_pre_ping=True
)


In [85]:
Lap.to_sql('lapdata',con=engine,index=False, if_exists='replace')

418

In [80]:
from urllib.parse import quote
pw_original='Jesraa@11'0
pw_encode=quote(pw_original)
print(pw_encode)

Jesraa%4011
